# End-to-end run no GPU

### Imports

In [1]:
# -- public imports

from transformers import BigBirdTokenizer, AutoTokenizer, AutoModelForTokenClassification, AutoConfig
import pandas as pd
from torch.utils.data import DataLoader
import torch

In [38]:
# -- dev imports
# TODO prediction string stasrt from 0 or 1?
%load_ext autoreload
%autoreload 2

from argminer.data import ArgumentMiningDataset
from argminer.evaluation import inference, get_word_labels

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
path_label_map = '../../data/UCL/dataset2/df_label_map_bio.json'
path_dataset = '../../data/UCL/dataset2/df_labels_general.json'

df_label_map = pd.read_json(path_label_map)
df_text = pd.read_json(path_dataset)
df_text.predictionString = df_text.predictionString.apply(lambda x: ' '.join([str(item) for item in x]))
df_text.label_bio = df_text.label_bio.apply(lambda x: ' '.join([str(item) for item in x]))
df_text.head()

,label_type,label,text,span_start,span_end,doc_id,predictionString,label_bio,label_bieo
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001,83 84 85 86 87 88 89 90 91 92,B-MajorClaim I-MajorClaim I-MajorClaim I-Major...,"[B-MajorClaim, I-MajorClaim, I-MajorClaim, I-M..."
2,O,Other,".\nFirst of all,",575,591,essay001,93 94 95 96,O O O O,"[O, O, O, O]"
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001,97 98 99 100 101 102 103 104 105 106 107 108 1...,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...,"[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ..."
4,O,Other,.,714,716,essay001,115,O,[O]


0        1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1                            83 84 85 86 87 88 89 90 91 92
2                                              93 94 95 96
3        97 98 99 100 101 102 103 104 105 106 107 108 1...
4                                                      115
                               ...                        
12575                                        137293 137294
12576    137295 137296 137297 137298 137299 137300 1373...
12577                                 137305 137306 137307
12578    137308 137309 137310 137311 137312 137313 1373...
12579                                               137320
Name: predictionString, Length: 12580, dtype: object

In [87]:
df_input = df_text.groupby('doc_id').agg({
    'text':lambda x: ' '.join(x),
    'predictionString': lambda x: ' '.join(x),
    'label_bio': lambda x: ' '.join(x)
})
df_input.label_bio = df_input.label_bio.str.split()
df_input

,text,predictionString,label_bio
doc_id,,,
essay001,Should students be taught to compete or to coo...,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay002,More people are migrating to other countries t...,388 389 390 391 392 393 394 395 396 397 398 39...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay003,International tourism is now more common than ...,734 735 736 737 738 739 740 741 742 743 744 74...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay004,International tourism is now more common than ...,1004 1005 1006 1007 1008 1009 1010 1011 1012 1...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay005,Living and studying overseas\n\nIt is every st...,1303 1304 1305 1306 1307 1308 1309 1310 1311 1...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...,...
essay398,We can not forcedly put the same numbers of ma...,135846 135847 135848 135849 135850 135851 1358...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay399,"Drugs, alcohol and messy sex lives\n\nCelebrit...",136187 136188 136189 136190 136191 136192 1361...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
essay400,A greater proportion of the budget should be a...,136459 136460 136461 136462 136463 136464 1364...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [28]:
model_name = 'google/bigbird-roberta-base'
num_labels = df_label_map.shape[0]

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)
tokenizer = AutoTokenizer.from_pretrained(model_name)
optimizer = torch.optim.Adam(params=model.parameters())


Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForTokenClassification were no

In [88]:
limit_size = 2

dataset = ArgumentMiningDataset(
    df_label_map,
    df_input[['text', 'label_bio']].rename(columns={'label_bio':'labels'}),
    tokenizer,
    max_length=256,
    strategy='standard_bio'
)

trainloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [102]:
epochs = 2
for epoch in range(epochs):
    training_loss = 0
    for i, (inputs, targets) in enumerate(trainloader):
        optimizer.zero_grad()

        loss, outputs = model(
            labels=targets,
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=False
        )
            
        training_loss += loss.item()

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        break
    print(training_loss/(i+1))

1.6381123065948486
1.5479151010513306


In [130]:
val_dataset = ArgumentMiningDataset(
    df_label_map,
    df_input[['text', 'label_bio']].rename(columns={'label_bio':'labels'}),
    tokenizer,
    max_length=512,
    strategy='standard_bio',
    is_train = False
)
valloader = DataLoader(val_dataset, batch_size=2, shuffle=False)
print(df_input)

                                                       text  \
doc_id                                                        
essay001  Should students be taught to compete or to coo...   
essay002  More people are migrating to other countries t...   
essay003  International tourism is now more common than ...   
essay004  International tourism is now more common than ...   
essay005  Living and studying overseas\n\nIt is every st...   
...                                                     ...   
essay398  We can not forcedly put the same numbers of ma...   
essay399  Drugs, alcohol and messy sex lives\n\nCelebrit...   
essay400  A greater proportion of the budget should be a...   
essay401  Fatherhood should be as present as motherhood ...   
essay402  Children should studying hard or playing sport...   

                                           predictionString  \
doc_id                                                        
essay001  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18

In [339]:
def evaluate(df_outputs, df_targets):
    # TODO problem of duplicates? Think about this with test cases
    """
    Calculates the F-Score given prediction strings
    """
    # -- Constants
    gt, pred = '_gt', '_pred'
    
    #    Merge constants
    merge_on = ['id', 'class']
    suffixes = (gt, pred)

    #    Columns labels
    overlap_pred_gt = f'overlap{pred}{gt}'  # how much of ground truth in prediction
    overlap_gt_pred = f'overlap{gt}{pred}'  # how much of prediction in ground truth
    pred_col_name = 'predictionString'
    gt = f'{pred_col_name}{gt}'
    pred = f'{pred_col_name}{pred}'

    
    # TODO is this even correct?? Would you not be repeating items??
    # find all combinations
    # REally really not sure of this assumption...
    df_targets = df_targets.merge(
        df_outputs, on=merge_on, suffixes=suffixes
    )    
    # find intersection and normalise against each item
    df_targets[overlap_pred_gt] = df_targets[[gt, pred]].apply(
        lambda x: len(x[gt].intersection(x[pred])) / len(x[pred]), axis=1
    )
    
    df_targets[overlap_gt_pred] = df_targets[[pred, gt]].apply(
        lambda x: len(x[gt].intersection(x[pred])) / len(x[gt]), axis=1
    )
    

    df_targets['tp'] = df_targets[[overlap_pred_gt, overlap_gt_pred]].apply(
        lambda x: int(x.overlap_pred_gt >= 0.5 and x.overlap_gt_pred >= 0.5), axis=1
    )
    df_targets['overlap_sum'] = df_targets.overlap_pred_gt + df_targets.overlap_gt_pred
    df_targets[pred] = df_targets[pred].apply(lambda x: tuple(x))
    df_targets[gt] = df_targets[gt].apply(lambda x: tuple(x))
    
    # only keep the ones with the highest overall score
    df_targets_match = df_targets[df_targets['tp'] == 1].groupby(
        ['id', 'class', pred]
    ).agg({'overlap_sum':'max', gt: lambda x: x})
    
    # need to find TP by class!
    TP = df_targets_match.shape[0]
    
    # find the items that have no match, but also do not have predictionStrings that are matched
    df_targets_no_match = df_targets[
        (df_targets['tp'] == 0) & (~df_targets.set_index(['id', 'class', pred]).index.isin(df_targets_match.index))
    ]


    
    ids_no_matches = (df_targets_no_match.overlap_pred_gt == 0) & (df_targets_no_match.overlap_gt_pred == 0)
    # filter out the false positives
    df_false_positives = df_targets_no_match[
        ~ids_no_matches
    ]


    df_false_negatives = df_targets_no_match[
        (ids_no_matches) & (~df_targets_no_match.set_index(['id', 'class', gt]).index.isin(
            df_targets_match.reset_index().set_index(['id', 'class', gt]).index
        ))
    ]
    
    TP = df_targets_match.reset_index().groupby(['id', 'class']).size().to_frame('tp').reset_index()
    FN = df_false_negatives.groupby(['id', 'class']).size().to_frame('fn').reset_index()
    FP = df_false_positives.groupby(['id', 'class']).size().to_frame('fp').reset_index()
    
    scores = TP.merge(
        FN.merge(FP, how='outer'), how='outer'
    ).fillna(0)
    
    
    # TODO need to apply f1 score
    return scores

In [340]:
df_outputs = pd.DataFrame(data={
    'id':[1,1,2,1],
    'class':[1,1,1, 2],
    'predictionString':[{1,2}, {6,7}, {1,2,3}, {1,2}]
})
df_targets = pd.DataFrame(data={
    'id':[1,1,1,2, 1],
    'class':[1,1,1,1, 2],
    'predictionString':[{1,2,3, 4, 5}, {6,7,8}, {21,22,23,24,25}, {1,2,3}, {1,2}]
})

evaluate(df_outputs, df_targets)

,id,class,tp,fn,fp
0,1,1,1,1.0,1.0
1,1,2,1,0.0,0.0
2,2,1,1,0.0,0.0


In [207]:
with torch.no_grad():
    
    for i, (inputs, targets) in enumerate(valloader):
        loss, outputs = model(
            labels=targets,
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=False
        )
        
        # validation should have this, all can still be on the GPU
        word_ids = inputs['word_ids']
        doc_ids = inputs['index']
        
        word_label_probas = get_word_labels(word_ids, outputs, agg_strategy='first', has_x=False)
        # can still be GPU
        word_labels = [tensor.argmax(dim=1) for tensor in word_label_probas]
        # still GPU
        
        target_labels = get_word_labels(word_ids, targets, agg_strategy='first', has_x=False)
        
        
        # todo need to have a label to map groups of items back to each other,
        # so first convert back to strings, then relabel based on strings, e.g. remove the 'B-I' stuff!
        df_targets_predString = get_predictionString(target_labels, doc_ids)
        df_outputs_predString = get_predictionString(word_labels, doc_ids)
        
        evaluate(df_outputs_predString, df_targets_predString)
        
        # also needs the doc ID for the mapping, so doc_id is missing!!!!
        break

   id  class                                   predictionString
0   0      0  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
1   1      0  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...
    id  class                                predictionString_gt  \
0    0      0  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
1    0      0  {38828, 38829, 38830, 38831, 38832, 38833, 388...   
2    0      0  {48115, 48116, 48117, 48118, 48119, 48120, 481...   
3    0      0  {59512, 59513, 59514, 59515, 59516, 59517, 595...   
4    0      0  {75776, 75777, 75778, 75779, 75780, 75781, 757...   
5    0      0  {84838, 84839, 84840, 84841, 84842, 84843, 848...   
6    0      0  {93281, 93282, 93283, 93284, 93285, 93286, 932...   
7    0      0  {98770, 98771, 98772, 98773, 98774, 98775, 987...   
8    0      0  {108057, 108058, 108059, 108060, 108061, 10806...   
9    0      0  {137178, 137179, 137180, 137181, 137182, 13718...   
10   0      0  {147731, 147732, 147733, 147734, 147735, 1477

TypeError: unhashable type: 'set'

In [144]:
df_text[df_text.doc_id == 'essay001']

,label_type,label,text,span_start,span_end,doc_id,predictionString,label_bio,label_bieo
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001,83 84 85 86 87 88 89 90 91 92,B-MajorClaim I-MajorClaim I-MajorClaim I-Major...,"[B-MajorClaim, I-MajorClaim, I-MajorClaim, I-M..."
2,O,Other,".\nFirst of all,",575,591,essay001,93 94 95 96,O O O O,"[O, O, O, O]"
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001,97 98 99 100 101 102 103 104 105 106 107 108 1...,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...,"[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ..."
4,O,Other,.,714,716,essay001,115,O,[O]
5,T4,Premise,What we acquired from team work is not only ho...,716,851,essay001,116 117 118 119 120 121 122 123 124 125 126 12...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
6,O,Other,.,851,853,essay001,142,O,[O]
7,T5,Premise,"During the process of cooperation, children ca...",853,1086,essay001,143 144 145 146 147 148 149 150 151 152 153 15...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
8,O,Other,.,1086,1088,essay001,180,O,[O]
9,T6,Premise,All of these skills help them to get on well w...,1088,1191,essay001,181 182 183 184 185 186 187 188 189 190 191 19...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."


In [157]:
def get_predictionString(labels, doc_ids):
    """ 
    """
    # TODO need to ensure that we are mapping to the correct class names and doc IDs!
    ids = []
    classes = []
    prediction_strings = []
    for doc_id, label in zip(doc_ids, labels):
        unique_classes, unique_class_counts = torch.unique_consecutive(label, return_counts=True)
        start_id = 1
        for unique_class, unique_class_count in zip(unique_classes, unique_class_counts):
            ids.append(doc_id.item())
            classes.append(unique_class.item())
            end_id = start_id + unique_class_count + 1 
            prediction_strings.append(set(range(start_id, end_id)))
            start_id = end_id
    return pd.DataFrame(data={'id': ids, 'class': classes, 'predictionString': prediction_strings})

In [127]:
df_text[df_text.doc_id == 'essay001']

,label_type,label,text,span_start,span_end,doc_id,predictionString,label_bio,label_bieo
0,O,Other,Should students be taught to compete or to coo...,0,503,essay001,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,O O O O O O O O O O O O O O O O O O O O O O O ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,T1,MajorClaim,we should attach more importance to cooperatio...,503,575,essay001,83 84 85 86 87 88 89 90 91 92,B-MajorClaim I-MajorClaim I-MajorClaim I-Major...,"[B-MajorClaim, I-MajorClaim, I-MajorClaim, I-M..."
2,O,Other,".\nFirst of all,",575,591,essay001,93 94 95 96,O O O O,"[O, O, O, O]"
3,T3,Claim,"through cooperation, children can learn about ...",591,714,essay001,97 98 99 100 101 102 103 104 105 106 107 108 1...,B-Claim I-Claim I-Claim I-Claim I-Claim I-Clai...,"[B-Claim, I-Claim, I-Claim, I-Claim, I-Claim, ..."
4,O,Other,.,714,716,essay001,115,O,[O]
5,T4,Premise,What we acquired from team work is not only ho...,716,851,essay001,116 117 118 119 120 121 122 123 124 125 126 12...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
6,O,Other,.,851,853,essay001,142,O,[O]
7,T5,Premise,"During the process of cooperation, children ca...",853,1086,essay001,143 144 145 146 147 148 149 150 151 152 153 15...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."
8,O,Other,.,1086,1088,essay001,180,O,[O]
9,T6,Premise,All of these skills help them to get on well w...,1088,1191,essay001,181 182 183 184 185 186 187 188 189 190 191 19...,B-Premise I-Premise I-Premise I-Premise I-Prem...,"[B-Premise, I-Premise, I-Premise, I-Premise, I..."


In [ ]:
def get_predictionString(labels, doc_ids):
    for label in labels:
        unique_classes, unique_class_counts = torch.unique_consecutive(return_counts=True)
        

In [99]:
df_input.iloc[[0,1]].label_bio.apply(lambda x: len(x))

doc_id
essay001    387
essay002    346
Name: label_bio, dtype: int64

,0,1
0,1,2
1,3,4
